In [1]:
from itertools import groupby
from operator import itemgetter

In [1]:
# this notebook is dedicated to parsing a list of novel isoforms (exon-chains)
# and seeing if anything is wrong with the start codon - missing or not

In [9]:
def startTrimming(orf,chain,strand):
    if strand=="+":
        tmp=orf.split("-")
        orfStarts=[int(x) for x in tmp[0].split(",")]
        orfEnds=[int(x) for x in tmp[-1].split(",")]
        
        print(orfStarts,orfEnds)
        # next we need to parse the exons in the same manner
        tmp=chain.split("-")
        chainStarts=[int(x) for x in tmp[0].split(",")]
        chainEnds=[int(x) for x in tmp[-1].split(",")]

        orf=[]
        for t in zip(orfStarts,orfEnds):
            orf+=list(range(t[0],t[1]+1))

        chain=[]
        for t in zip(chainStarts,chainEnds):
            chain+=list(range(t[0],t[1]+1))
        
        print(orf,chain)
        # now remove anything in the chain that is before or after the start and end of the ORF
        chain=[x for x in chain if x>=orf[0] and x<=orf[-1]]
        print(chain)
        
        if len(chain)<3: # should hopefully never happen
            return 7777777

        setStartCodon_orf=set(orf[:3])
        setStartCodon_chain=set(chain[:3])
        startCodon_orf_diff_chain=setStartCodon_orf-setStartCodon_chain
        print(startCodon_orf_diff_chain)
        if orf[0]==chain[0]: # at least the first nucleotide is there
            if len(startCodon_orf_diff_chain)<3: # at least part of the start codon is there
                return len(startCodon_orf_diff_chain) # return the number of missing nucleotides

        # what we shall do instead is simply quantify how far down the orf the exon begins
        firstExonChainNT=min(chain)
        curPos=0 # position within the ORF
        orfIter=iter(orf)
        while not next(orfIter)>=firstExonChainNT:
            curPos+=1
        return curPos
    else:
        tmp=orf.split("-")
        orfStarts=[int(x) for x in tmp[0].split(",")]
        orfEnds=[int(x) for x in tmp[-1].split(",")]
        
        print(orfStarts,orfEnds)
        tmp=chain.split("-")
        chainStarts=[int(x) for x in tmp[0].split(",")]
        chainEnds=[int(x) for x in tmp[-1].split(",")]
        
        orf=[]
        for t in zip(orfStarts,orfEnds):
            orf+=list(range(t[0],t[1]+1))
        orf=orf[::-1]
        
        chain=[]
        for t in zip(chainStarts,chainEnds):
            chain+=list(range(t[0],t[1]+1))
        chain=chain[::-1]
        
        print(orf,chain)
        chain=[x for x in chain if x<=orf[0] and x>=orf[-1]]
        print(chain)
        
        if len(chain)<3: # should hopefully never happen
            return 7777777

        setStartCodon_orf=set(orf[:3])
        setStartCodon_chain=set(chain[:3])
        startCodon_orf_diff_chain=sorted(list(setStartCodon_orf-setStartCodon_chain))[::-1]
        print(startCodon_orf_diff_chain)
        if orf[0]==chain[0]: # at least the first nucleotide is there
            if len(startCodon_orf_diff_chain)<3: # at least part of the start codon is there
                return len(startCodon_orf_diff_chain) # return the number of missing nucleotides

        # what we shall do instead is simply quantify how far down the orf the exon begins
        firstExonChainNT=max(chain)
        curPos=0 # position within the ORF
        orfIter=iter(orf)
        while not next(orfIter)<=firstExonChainNT:
            curPos+=1
        return curPos

# orf="1,7-3,15"
# chain="3,9,15-4,9,18"
# orf="2,8-5,11"
# chain="3,8-5,9"
orf='118527495,118528603,118530654-118527612,118528791,118530916'
chain='118527496,118528502,118530654-118527612,118528791,118530982'
strand="+"
startTrimming(chain,orf,strand)

[118527496, 118528502, 118530654] [118527612, 118528791, 118530982]
[118527496, 118527497, 118527498, 118527499, 118527500, 118527501, 118527502, 118527503, 118527504, 118527505, 118527506, 118527507, 118527508, 118527509, 118527510, 118527511, 118527512, 118527513, 118527514, 118527515, 118527516, 118527517, 118527518, 118527519, 118527520, 118527521, 118527522, 118527523, 118527524, 118527525, 118527526, 118527527, 118527528, 118527529, 118527530, 118527531, 118527532, 118527533, 118527534, 118527535, 118527536, 118527537, 118527538, 118527539, 118527540, 118527541, 118527542, 118527543, 118527544, 118527545, 118527546, 118527547, 118527548, 118527549, 118527550, 118527551, 118527552, 118527553, 118527554, 118527555, 118527556, 118527557, 118527558, 118527559, 118527560, 118527561, 118527562, 118527563, 118527564, 118527565, 118527566, 118527567, 118527568, 118527569, 118527570, 118527571, 118527572, 118527573, 118527574, 118527575, 118527576, 118527577, 118527578, 118527579, 1185275

0

In [ ]:
# now we also need to figure out a way to verify that the extension will not alter another exon here

# The goal is to make sure that the transcripts can indeed be safely extended, without interfering with another exon

# if a prior exon is found, then we shall report it as such, and see if perhaps the exons should be merged

# however this should not happen by the definitions we've created for these isoforms in the trimming function
# which guarantees that all bases equal or greater in value than the first and last bases of the orf are guaranteed
# to be present in the orf. By this definition, all exons should already exist in there

In [2]:
def trimming(orf,chain,strand,parent,start=True):
    if not start:
        if strand=="+":
            strand="-"
        else:
            strand="+"
    if strand=="+":
        tmp=orf.split("-")
        orfStarts=[int(x) for x in tmp[0].split(",")]
        orfEnds=[int(x) for x in tmp[-1].split(",")]

        # next we need to parse the exons in the same manner
        tmp=chain.split("-")
        chainStarts=[int(x) for x in tmp[0].split(",")]
        chainEnds=[int(x) for x in tmp[-1].split(",")]

        orf=[]
        for t in zip(orfStarts,orfEnds):
            orf+=list(range(t[0],t[1]+1))

        chain=[]
        for t in zip(chainStarts,chainEnds):
            chain+=list(range(t[0],t[1]+1))

        # now remove anything in the chain that is before or after the start and end of the ORF
        chain=[x for x in chain if x>=orf[0] and x<=orf[-1]]

        if len(chain)<3: # should hopefully never happen
            return 7777777,7777777

        setStartCodon_orf=set(orf[:3])
        setStartCodon_chain=set(chain[:3])
        startCodon_orf_diff_chain=setStartCodon_orf-setStartCodon_chain

        if orf[0]==chain[0]: # at least the first nucleotide is there
            if len(startCodon_orf_diff_chain)<3: # at least part of the start codon is there
                return len(startCodon_orf_diff_chain),0 # return the number of missing nucleotides

        # what we shall do instead is simply quantify how far down the orf the exon begins
        firstExonChainNT=min(chain)
        curPos=0 # position within the ORF    
        orfIter=iter(orf)
        skippedExonBases=0
        exons=0
        prevOrf=None
        curOrf=next(orfIter)
        while True:
            if not curOrf>=firstExonChainNT:
                if prevOrf and not curOrf-prevOrf==1:
                    if skippedExonBases==0:
                        skippedExonBases+=curPos
                    exons+=1
                curPos+=1
                prevOrf=curOrf
                curOrf=next(orfIter)
            else:
                if prevOrf and not curOrf-prevOrf==1:
                    if skippedExonBases==0:
                        skippedExonBases+=curPos
                    exons+=1
                break
        return curPos,exons,skippedExonBases
    else:
        tmp=orf.split("-")
        orfStarts=[int(x) for x in tmp[0].split(",")]
        orfEnds=[int(x) for x in tmp[-1].split(",")]
        
        tmp=chain.split("-")
        chainStarts=[int(x) for x in tmp[0].split(",")]
        chainEnds=[int(x) for x in tmp[-1].split(",")]
        
        orf=[]
        for t in zip(orfStarts,orfEnds):
            orf+=list(range(t[0],t[1]+1))
        orf=orf[::-1]
        
        chain=[]
        for t in zip(chainStarts,chainEnds):
            chain+=list(range(t[0],t[1]+1))
        chain=chain[::-1]
        
#         print(chain)
#         print(orf)
        chain=[x for x in chain if x<=orf[0] and x>=orf[-1]]
            
        if len(chain)<3: # should hopefully never happen
            return 7777777,7777777
        
        setStartCodon_orf=set(orf[:3])
        setStartCodon_chain=set(chain[:3])
        startCodon_orf_diff_chain=sorted(list(setStartCodon_orf-setStartCodon_chain))[::-1]

        if orf[0]==chain[0]: # at least the first nucleotide is there
            if len(startCodon_orf_diff_chain)<3: # at least part of the start codon is there
                return len(startCodon_orf_diff_chain),0 # return the number of missing nucleotides

        # what we shall do instead is simply quantify how far down the orf the exon begins
        firstExonChainNT=max(chain)
        curPos=0 # position within the ORF
        orfIter=iter(orf)
        exons=0
        skippedExonBases=0
        prevOrf=None
        curOrf=next(orfIter)
        while True:
            if not curOrf<=firstExonChainNT:
                if prevOrf and not prevOrf-curOrf==1:
                    if skippedExonBases==0:
                        skippedExonBases+=curPos
                    exons+=1
                curPos+=1
                prevOrf=curOrf
                curOrf=next(orfIter)
            else:
                if prevOrf and not prevOrf-curOrf==1:
                    if skippedExonBases==0:
                        skippedExonBases+=curPos
                    exons+=1
                break
        return curPos,exons,skippedExonBases


orf='24573637,24574245,24574675,24575265,24576169-24573810,24574499,24574810,24575412,24576223'
# chain='24574245,24574675,24575151,24576169-24574499,24574810,24575744,24576220'
# chain='24573518,24574245,24574675,24575151-24573835,24574499,24574810,24575744'
chain='24573637,24574245,24574675,24575265-24573810,24574499,24574810,24575402'
strand="-"
trimming(orf,chain,strand,"csd",True)

(65, 1, 55)

In [4]:
# another interesting evaluation to perform is to check for transcripts that are outside
# known start/end coordinates as annotated by GENCODE and RefSeq
# two such were already found by manual analysis: CHS.36728.3, CHS.51125.3

def outOfBounds(orfList,chain):
#     orfList=orfs.split(";")
    starts=[]
    ends=[]
    for orf in orfList:
        # get start/end
        tmp=orf.split("-")
        orfStarts=[int(x) for x in tmp[0].split(",")]
        orfEnds=[int(x) for x in tmp[1].split(",")]
        starts.append(sorted(orfStarts)[0])
        ends.append(sorted(orfEnds)[-1])
    minOrfStart=min(starts)
    maxOrfEnd=max(ends)
    print(minOrfStart,maxOrfEnd)
    
    # next we need to parse the exons in the same manner
    tmp=chain.split("-")
    chainStarts=[int(x) for x in tmp[0].split(",")]
    minChainStart=min(chainStarts)
    maxChainStart=max(chainStarts)
    chainEnds=[int(x) for x in tmp[-1].split(",")]
    minChainEnd=min(chainEnds)
    maxChainEnd=max(chainEnds)
    print(minChainEnd,maxChainEnd)

    if maxChainEnd<minOrfStart or minChainStart>maxOrfEnd:
        return True
    return False
    
        
orfs=['7025085,7042980,7063897,7070196,7070720,7071560,7071733,7074560,7080732,7085898,7086290,7087267,7090089,7091472-7025142,7043125,7064149,7070328,7070861,7071623,7071830,7074620,7080853,7085972,7086417,7087432,7090190,7091492', '7025085,7042980,7063897,7070196,7070720,7071560,7071733,7074560,7080732,7085898,7086290,7087267,7090089,7100785-7025142,7043125,7064149,7070328,7070861,7071623,7071830,7074620,7080853,7085972,7086417,7087432,7090190,7100970', '7025085,7042980,7063897,7070196,7070720,7071560,7071733,7074560,7080732,7085898,7086290,7087267,7090089,7100785,7105398-7025142,7043125,7064149,7070328,7070861,7071623,7071830,7074620,7080853,7085972,7086417,7087432,7090190,7100850,7105406', '7042880,7063897,7070196,7070720,7071560,7071733,7074560,7080732,7085898,7086290,7087267,7090089,7091472-7043125,7064149,7070328,7070861,7071623,7071830,7074620,7080853,7085972,7086417,7087432,7090190,7091492']
chain="6910697,6912088,6914579,6915592-6910982,6912172,6914678,6924428"
outOfBounds(orfs,chain)

7025085 7105406
6910982 6924428


True